In [17]:
load('Productos c-d.py')
load('g2.py')
load('O to z.py')

Vamos a utilizar el paper de Milnor donde se muestran fórmulas más sencillas para el cálculo de curvaturas. En dicho paper se trabaja sobre bases ortonormales en el álgebra de Lie, por lo que calcularemos una para $\mathfrak g_2$ con el producto interno $\langle\,,\rangle^*$.

#### Cargamos una base ortonormal de $\mathfrak g_2$ para $\langle\,,\rangle^*$

Consideremos la base ortonormal de $T_{p_0}\mathscr Z$ que conseguimos aplicando el proceso de Gram-Schmidt a la base dada por $\{D_0\cdot p_0,\dots,D_{13}\cdot p_0\}$ donde $\{D_0,\dots,D_{13}\}$ es la base de $\mathfrak g_2$ que es ortonormal para la métrica bi-invariante.

In [18]:
p0 = octs_to_Z(o_basis[1],o_basis[2],o_basis[4],-o_basis[7])
Tp0Z_basis = [act_g2_Z(A,p0) for A in g2_basis]
from sage.modules.misc import gram_schmidt
Tp0Z_BO = gram_schmidt(Tp0Z_basis)[0]
M1 = matrix(14,14,[v*w for v in Tp0Z_BO for w in Tp0Z_BO])

Tp0Z_BON = [Tp0Z_BO[i]*(1/sqrt(M1[i,i])) for i in range(14)]

Obtenemos las matrices $A_i \in\mathfrak g_2$ tales que $A_i \cdot p_0$ es el $i$-ésimo elemento de la base ortonormal de $T_{p_0}\mathscr Z$.

In [19]:
def A(k,TP_base):
    """
    devuelve la matriz de g2 asociada al elemento k-ésimo de la base TP_base
    """
    #primer sistema
    A11 = D_g2_gen * o_basis[1]
    A11_eqs = flatten([(A11 - Z_to_octs(TP_base[k])[0]).list()])
    B1 = D_g2_gen.subs(solve(A11_eqs, D.list(),algorithm='sympy'))
    #segundo sistema
    A12 = B1 * o_basis[2]
    A12_eqs = flatten([(A12 - Z_to_octs(TP_base[k])[1]).list()])
    B2 = B1.subs(solve(A12_eqs, D.list(),algorithm='sympy'))
    #tercer sistema
    A13 = B2 * o_basis[4]
    A13_eqs = flatten([(A13 - Z_to_octs(TP_base[k])[2]).list()])
    B3 = B2.subs(solve(A13_eqs, D.list(),algorithm='sympy'))
    #cuarto sistema
    A14 = - B3 * o_basis[7] 
    A14_eqs = flatten([(A14 - Z_to_octs(TP_base[k])[3]).list()])
    A = B3.subs(solve(A14_eqs, D.list(),algorithm='sympy'))
    return A

g2_BON = [A(i,Tp0Z_BON) for i in range(14)]

Así, por la definición de la métrica $\langle,\rangle$ tenemos que esta base $\{A_0,\dots,A_{13}\}$ es una base ortonormal para la métrica $\langle\,,\rangle^*$.

Estas bases ortonormales de $T_{p_0}\mathscr Z$ y $\mathfrak g_2$ quedan cargadas en las listas `Tp0Z_BON` y `g2_BON`.

# Curvatura seccional

En $G_2$ tenemos una métrica definida por el pullback de la métrica en $\mathscr{Z}$ heredada de $\mathbb R^{32}$, la cual denotaremos por $\langle,\rangle^*$. Nuestro objetivo ahora es calcular la curvatura seccional de $G_2$ para esta métrica.

Para vectores ortonormales $u,v$ la curvatura seccional está dada por

$$K(u,v) = \langle R(u,v)u, v\rangle$$

En el trabajo de Milnor se prueba lo siguiente

Dado un grupo de Lie con una métrica invariante a izquierda, $(G,\langle,\rangle)$, para cualquier base ortonormal de $\mathfrak g$ y con coeficientes de estructura $\alpha_{ijk}$, la curvatra seccional $K(e_i,e_j)$ está dada por  

\begin{align*}
K(e_i,e_j) &= \sum_{k} \Big( \tfrac{1}{2} \alpha_{ijk}\big(-\alpha_{ijk} + \alpha_{jki} + \alpha_{kij}\big) \\
& - \tfrac{1}{4}\big(\alpha_{ijk}-\alpha_{jki}+\alpha_{kij}\big)\big(\alpha_{ijk}+\alpha_{jki}-\alpha_{kij}\big) \\
& - \alpha_{kii}\alpha_{kjj}\Big)
\end{align*}

 Armamos un programa para calcular los coeficientes $\alpha_{ijk} = \langle[A_i,A_j],A_k\rangle^*$.

In [20]:
def alpha(i,j,k):
    """
    función que recive números i,j,k entre 0 y 13 y devuelve el coeficiente alfa_{ijk} en la base ortonormal de g_2, 
    g2_BON
    """
    H = bracket(g2_BON[i],g2_BON[j])
    h = act_g2_Z(H,p0)
    return h*Tp0Z_BON[k]

alphas = {(i,j,k): alpha(i,j,k) for i in range(14) for j in range(14) for k in range(14)}

Con esto y la fórmula expresada antes, calculamos la curvatura seccional entre elementos de la base ortonormal como sigue:

In [21]:
def curv_sec(i,j):
    h = sum(1/2*alphas[(i,j,k)]*(-alphas[(i,j,k)]+alphas[(j,k,i)]+alphas[(k,i,j)]) - 1/4*(alphas[(i,j,k)]-alphas[(j,k,i)]+alphas[(k,i,j)])*(alphas[(i,j,k)]+alphas[(j,k,i)]-alphas[(k,i,j)])-alphas[(k,i,i)]*alphas[(k,j,j)] for k in range(14))
    return h   

In [22]:
[curv_sec(i,0)._sympy_().simplify()._sage_() for i in range(13)]

[0, 1/2, 1/2, 0, 1/8, 1/6, 1/8, 1/6, 0, 1/6, -1/8, 1/6, -1/8]

Como se puede observar, existen casos donde la curvatura seccional es negativa.

## Curvatura de Ricci

La *curvatura de Ricci*, $Ric(Y,Z)$, se define como la traza de la transformación 

$$X \mapsto R(X,Y)Z.$$

Como $R$ es lineal en las tres variables, resulta que la aplicación definida antes es una aplicación lineal de $\mathfrak g \rightarrow \mathfrak g$ por lo que se puede representar como una matriz respecto a una base ortonormal $\{e_0,e_1,\dots,e_n\}$ de $\mathfrak g$. Entonces, para esta base se tiene que la curvatura de Ricci resulta

$$Ric(Y,Z) = \sum_i \langle R(e_i,Y)Z, e_i \rangle.$$

Definimos la *curvatura de Ricci en la dirección $X$*

$$r(X) = Ric(X,X) = \sum_{i}\langle R(e_i,X)X, e_i \rangle$$

para $X \in \mathfrak g$ unitario.

Una variedad riemanniana se dice una *variedad de Einstein* si existe una constante $\lambda \in \mathbb R$ tal que $$Ric(X,Y) = \lambda \langle X,Y\rangle.$$ 

Observemos que $Ric : \mathfrak g \times \mathfrak g \rightarrow \mathbb R$ es una aplicación bilineal, por lo que considerando una base ortonormal es posible represntarla a través una matriz $[Ric]$ donde la entrada $ij$ es $Ric(e_i,e_j)$. Entonces, para ver si un variedad riemanniana $G$ es Einstein, sólo debemos verificar si $$[Ric] = \lambda Id$$ para algún $\lambda \in \mathbb R$.

Nuestro objetivo ahora es ver si $(G_2,\langle\,,\rangle^*)$ es una variedad de Einstein. Entonces, considerando la base ortonormal $\{A_0,A_1,\dots,A_{13}\}$ en $\mathfrak g_2$ tenemos 

$$Ric(Y,Z) = \sum_{i=0}^{13} \langle R(A_i,Y)Z, A_i \rangle^*$$

y

$$r(X) = \sum_{i}\langle R(A_i,X)X, A_i \rangle^*.$$

Para poder ambas curvaturas, primero definamos una forma genérica para el tensor de curvatura para $A_i,Y,Z$. Recordemos que

$$R(A_i,Y)Z =  \nabla_{[A_i,Y]}Z + \nabla_Y\nabla_{A_i} Z -\nabla_X \nabla_{A_i} Z.$$

Si $Y = \sum_{i=0}^{13} Y_i \cdot A_i$ y $Z = \sum_{i=0}^{13} Z_i \cdot A_i$ entonces

$$\nabla_{[A_i,Y]}Z = \nabla_{[A_i, \sum_{j=0}^{13} Y_j \cdot A_j]}\big(\sum_{k=0}^{13} Z_k \cdot e_k\big) = \sum_{j=0}^{13} Y_j \Big(\sum_{k=1}^{14} Z_k \cdot \nabla_{[A_i,A_j]}A_k \Big)$$

$$\nabla_Y\nabla_{A_i} Z = \nabla_{\sum_{j=0}^{13} Y_j \cdot A_j}\nabla_{A_i}\big(\sum_{k=0}^{13} Z_k \cdot A_k\big) = \sum_{j=0}^{13} Y_j \Big(\sum_{k=1}^{14} Z_k \cdot \nabla_{A_j}\nabla_{A_i}A_k \Big)$$

$$\nabla_{A_i}\nabla_Y Z = \nabla_{A_i}\nabla_{\sum_{j=0}^{13} Y_j \cdot A_j}\big(\sum_{k=0}^{13} Z_k \cdot A_k\big) = \sum_{j=0}^{13} Y_j \Big(\sum_{k=1}^{14} Z_k \cdot \nabla_{A_i}\nabla_{A_j}A_k \Big)$$

Entonces todo se resume a poder calcular $\nabla_{[A_i,A_j]}A_k$ y $\nabla_{A_j}\nabla_{A_i}A_k$ para todos $i,j,k \in \{0,1,\dots,13\}$. Para esto utilizamos un resultado que figura en el trabajo de Milnor para poder programar la conexión de Levi-Civita: si $\{e_0,\dots,e_{13}\}$ es una base ortonormal de $\mathfrak g_2$ entonces

$$\nabla_{e_i}e_j = \sum_k \tfrac{1}{2} (\alpha_{ijk}-\alpha_{jki}+\alpha_{kij}) e_k.$$

Definimos $\nabla_{A_i}A_j = \sum_k \tfrac{1}{2} (\alpha_{ijk}-\alpha_{jki}+\alpha_{kij}) A_k$ como sigue

In [23]:
def nabla_lc(i,j):
    """
    función que recibe números enteros i,j entre 0 y 13 y devuelve la conexión Levi-Civita de A_j en la dirección de
    A_i para A_i,A_j en la base ortonormal de g_2, g2_BON
    """
    h = sum((alphas[(i,j,k)]-alphas[(j,k,i)]+alphas[(k,i,j)])*g2_BON[k] for k in range(14))
    return 1/2*h

nabla_base = {(i,j):nabla_lc(i,j) for i in range(14) for j in range(14)}

Definimos ahora la conexión de Levi-Civita para dos elementos cualesquiera en $\mathfrak g_2$ usando la bilinealidad de la conexión

In [24]:
def levi_civita(A,B):
    """
    función que recibe dos matrices en g_2 y devuelve la conexión Levi-Civita de B en la dirección de A.
    Observación: Esta función no verifica que A y B sean elementos de g_2
    """
    X = act_g2_Z(A,p0)
    Y = act_g2_Z(B,p0)
    Y_coef = [Y*Tp0Z_BON[j] for j in range(14)]
    X_coef = [X*Tp0Z_BON[j] for j in range(14)]
    h = sum(X_coef[i]*sum((Y_coef[j])*nabla_base[(i,j)] for j in range(14)) for i in range(14))
    return h

Como queremos calcular $\nabla_{[A_i,A_j]}A_k$, cargamos un directorio con el corchete de elementos de la base

In [25]:
corch_base = {(i,j): g2_BON[i]*g2_BON[j]-g2_BON[j]*g2_BON[i] for i in range(14) for j in range(14)}

con esto definimos otro diccionario con la conexión Levi-Civita de un elemento de la base en la dirección del corchete entre dos elementos ($\nabla_{[A_i,A_j]}A_k$)

In [26]:
## Long time
## %time LC_corch_base = {(i,j,k) : levi_civita(corch_base[(i,j)],g2_BON[k]) for i in range(14) for j in range(14) for k in range(14)}
## save(LC_corch_base, "LC_corch_base")

como el diccionario anterior dura bastante tiempo, al resultado lo guardamos en un archivo .sobj y lo cargamos

In [27]:
LC_corch_base = load("LC_corch_base.sobj")

Y ahora definimos un último diccionario uno con $\nabla_{A_j}\nabla_{A_i}A_k$ para elementos de la base.

In [28]:
## Long time
##levi_civita_base = {(j,i,k) : levi_civita(g2_BON[j],nabla_base[(i,k)]) 
##                    for i in range(14) for j in range(14) for k in range(14)}

como el diccionario anterior dura bastante tiempo, al resultado lo guardamos en un archivo .sobj y lo cargamos

In [29]:
levi_civita_base = load("levi_civita_base.sobj")

Definimos ahora el tensor $R(A_i,Y)Z$ como sigue

In [30]:
def tensor_para_ricci(i,Y,Z):
    """
    función que recibe dos matrices de g_2 y un número i entre 0 y 13 y devuelve R(A_i,Y)Z donde e_u está
    en la base en la base ortonormal de g_2, g2_BON.
    Observación: Este programa no verifica que Y y Z sean elementos de g_2
    """
    y = act_g2_Z(Y,p0)
    z = act_g2_Z(Z,p0)
    y_coef = [y*Tp0Z_BON[j] for j in range(14)]
    z_coef = [z*Tp0Z_BON[j] for j in range(14)]
    nabla_corch = sum(y_coef[j] * sum(z_coef[k]*LC_corch_base[(j,i,k)] for k in range(14))for j in range(14))
    nabla_yiz =  sum(y_coef[j] * sum(z_coef[k]*levi_civita_base[(j,i,k)] for k in range(14))for j in range(14))
    nabla_iyz =  sum(y_coef[j] * sum(z_coef[k]*levi_civita_base[(i,j,k)] for k in range(14))for j in range(14))
    return nabla_corch + nabla_yiz - nabla_iyz

### Curvatura de Ricci

Definimos la curvatura de Ricci para $Y,Z$ en $\mathfrak g_2$

In [35]:
def curv_ricci(Y,Z):
    """
    Observación: este programa no verifica que X esté en g_2
    """
    tensor = [tensor_para_ricci(i,Y,Z) for i in range(14)]
    tensor_p0 = [act_g2_Z(tensor[i], p0) for i in range(14)]
    return sum(tensor_p0[i]*Tp0Z_BON[i] for i in range(14))

### Curvatura de Ricci en la dirección de $X$

Definimos la curvatura de Ricci en la dirección de $X$

In [37]:
def curv_dir_ricci(X):
    """
    Observación: este programa no verifica que X esté en g_2
    """
    tensor = [tensor_para_ricci(i,X,X) for i in range(14)]
    tensor_p0 = [act_g2_Z(tensor[i], p0) for i in range(14)]
    return sum(tensor_p0[i]*Tp0Z_BON[i] for i in range(14))

Creamos la lista con la curvatura de Ricci en la dirección de los vectores de la base ortonormal de $\mathfrak g_2$

In [38]:
#Long time
%time curv_ricci_base = [curv_dir_ricci(g2_BON[i]) for i in range(14)]

CPU times: user 16.8 s, sys: 109 ms, total: 16.9 s
Wall time: 17.8 s


In [40]:
curv_ricci_base_n = [curv_ricci_base[i]._sympy_().simplify()._sage_() for i in range(14)]
curv_ricci_base_n

[17/3,
 95/18,
 17/3,
 17/3,
 17/3,
 95/18,
 17/3,
 95/18,
 95/18,
 95/18,
 17/3,
 95/18,
 17/3,
 17/3]